# Data Wrangling - Users & Tweets

In [2]:
import os
import re
import datetime as dt
import time
import random
import json

# data science
import math
import numpy as np
import pandas as pd
import scipy.stats as st
from scipy.stats import randint as sp_randint

# data visualization
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from pyvis.network import Network

In [3]:
# configurations
# style for data visualizations
plt.style.use('fivethirtyeight')
%matplotlib inline
sns.set()

# Allow multiple outputs for each cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

# display all columns
pd.set_option('display.max_columns', None)

# suppress auto-conversion to scientific notation
pd.set_option('display.precision', 6)

In [107]:
tweets = pd.read_csv('Data/tweets2.csv', encoding='utf-8')
users = pd.read_csv('Data/users2.csv', encoding='utf-8')
hashtags = pd.read_csv('Data/hashtags.csv')

# Parsing geographical locations

In [5]:
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
}

states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

states_full = ['Alabama','Alaska','Arizona','Arkansas','California','Colorado',
                'Connecticut','Delaware','Florida','Georgia','Hawaii','Idaho', 
                'Illinois','Indiana','Iowa','Kansas','Kentucky','Louisiana',
                'Maine' 'Maryland','Massachusetts','Michigan','Minnesota',
                'Mississippi', 'Missouri','Montana','Nebraska','Nevada',
                'New Hampshire','New Jersey','New Mexico','New York',
                'North Carolina','North Dakota','Ohio',    
                'Oklahoma','Oregon','Pennsylvania','Rhode Island',
                'South  Carolina','South Dakota','Tennessee','Texas','Utah',
                'Vermont','Virginia','Washington','West Virginia',
                'Wisconsin','Wyoming']

places = ['San Francisco', 'Southern California', 'Chicago', 'Los Angeles', 'New York City', 'Brooklyn', 'Washington DC',
          'SF Peninsula', 'U.S.A', 'U.S.A.', 'U.S.', 'US', 'USA', 'United States', 'NYC', 'Seattle', 'Boston', 'Atlanta', 'Philadelphia']

In [6]:
geo = users['location'].str.split(r',', 1, expand=True)
geo.columns = ['cityOrState', 'country']
geo['cityOrState'] = geo['cityOrState'].str.strip()
geo['country'] = geo['country'].str.strip()
geo.head()

,cityOrState,country
0,Quelque part,None
1,London (E17) / Gateshead (NE9),None
2,add my snapchat - DirectPorn,None
3,NaN,NaN
4,NaN,NaN


In [7]:
index = (geo['cityOrState'].isin(places + states + states_full) | geo['country'].isin(places + states + states_full))

In [8]:
us = geo[index]

In [9]:
notus = geo[np.logical_not(index)]

In [10]:
len(us) + len(notus) == len(geo)

True

## US

1. remove '.' in country
2. 'US' and contains'United States' >> 'USA'
3. match 2 characters

In [11]:
us['country'] = us['country'].str.replace(r'.','')
us['cityOrState'] = us['cityOrState'].str.replace(r'.','')
us['country'] = us['country'].str.strip()

In [12]:
usa = ['USA','US','United States', 'United States of America']
us.loc[us['country'].str.contains('US', case=False) | us['country'].str.contains('United States', case=False), ['country']] = 'USA'
us[(us['cityOrState'].isin(usa) == True) & (us['country'].isin(usa) == False)]['country'].value_counts(dropna=False)

NaN                               6934
Earth                                3
Midwest                              2
Georgia                              2
Oklahoma                             2
Ohio                                 2
born & bred                          1
Europe                               1
Mexico & Canada                      1
Citizen of Earth                     1
Boston MA                            1
UK, All over the Globe               1
America                              1
midwest                              1
New York                             1
Louisiana                            1
where moss grows                     1
Middle East                          1
Canada, Europe                       1
Tunisia, Morocc                      1
India                                1
Iowa                                 1
Indiana                              1
EU                                   1
Ireland                              1
Puerto Rico, Vieques     

In [13]:
condition = (us['cityOrState'].isin(usa) == True) & (us['country'].isin(usa) == False)
condition2 = (us['cityOrState'].isin(usa) == True) & (us['country'].isin(usa) == True)
us.loc[condition, 'cityOrState'] = us.loc[condition, 'country']
us.loc[condition2, 'cityOrState'] = np.nan
us.loc[condition, 'country'] = 'USA'
us.loc[us['cityOrState'].str.contains('US', case=False) |
       us['cityOrState'].str.contains('United States', case=False), ['country']] = 'USA'
us.loc[us['cityOrState'].str.contains('US', case=False) |
       us['cityOrState'].str.contains('United States', case=False), ['cityOrState']] = np.nan
us['country'].value_counts(dropna=False)

USA                            23594
NaN                            10928
CA                              6382
NY                              4762
DC                              2625
FL                              1858
IL                              1545
TX                              1402
MA                              1395
PA                              1333
WA                              1305
GA                               992
NC                               944
VA                               774
AZ                               753
MI                               734
CO                               726
TN                               695
OH                               649
OR                               646
MN                               619
MD                               619
California                       580
MO                               558
NJ                               480
NV                               462
IN                               433
N

In [14]:
us.loc[us['country'].isna() == True, 'country'] = us.loc[us['country'].isna() == True, 'cityOrState']
us['country'].value_counts(dropna=False)

USA                        23594
CA                          6453
NY                          4906
DC                          2716
FL                          1896
IL                          1558
TX                          1437
MA                          1429
PA                          1379
WA                          1315
New York                    1295
California                  1116
Los Angeles                 1032
NC                          1017
GA                          1009
NYC                          833
VA                           807
AZ                           782
MI                           751
CO                           730
Texas                        724
TN                           718
New York City                705
Florida                      671
OH                           664
OR                           651
MN                           641
MD                           634
Chicago                      584
NJ                           578
          

In [15]:
ca = ['California', 'Los Angeles', 'San Francisco', 'LA', 'SF', 'Southern California', 'SF Peninsula']
us.loc[us['country'].isin(ca)|us['cityOrState'].isin(ca), 'cityOrState'] = 'CA'
us.loc[us['country'].isin(ca)|us['cityOrState'].isin(ca), 'country'] = 'USA'
us['country'].value_counts(dropna=False)

USA                      26624
CA                        6453
NY                        4906
DC                        2716
FL                        1896
IL                        1558
TX                        1437
MA                        1429
PA                        1379
WA                        1315
New York                  1295
NC                        1017
GA                        1009
NYC                        833
VA                         807
AZ                         782
MI                         751
CO                         730
Texas                      724
TN                         718
New York City              705
Florida                    671
OH                         664
OR                         651
MN                         641
MD                         634
Chicago                    584
NJ                         578
MO                         564
NV                         463
                         ...  
DC via Minnesota             1
you're m

In [16]:
ny = ['New York', 'NY', 'NYC', 'New York City', 'Brooklyn']
us.loc[us['country'].isin(ny)|us['cityOrState'].isin(ny), 'cityOrState'] = 'NY'
us.loc[us['country'].isin(ny)|us['cityOrState'].isin(ny), 'country'] = 'USA'
us['country'].value_counts(dropna=False)

USA                       34746
CA                         6452
DC                         2716
FL                         1895
IL                         1558
TX                         1437
MA                         1429
PA                         1379
WA                         1315
NC                         1017
GA                         1009
VA                          807
AZ                          782
MI                          751
CO                          730
Texas                       724
TN                          718
Florida                     671
OH                          663
OR                          651
MN                          641
MD                          634
Chicago                     581
NJ                          578
MO                          564
NV                          463
IN                          438
Ohio                        404
SC                          378
WI                          357
                          ...  
World   

In [17]:
us['country'] = us['country'].str.replace('U.S.A.', 'USA')
us['country'] = us['country'].str.replace(r'^U.S.$', 'USA')
us['country'] = us['country'].str.replace('Florida', 'FL')
us['country'] = us['country'].str.replace('Texas', 'TX')
us['country'] = us['country'].str.replace('Chicago', 'IL')
us['country'] = us['country'].str.replace('Illinois', 'IL')
us['country'] = us['country'].str.replace('Ohio', 'OH')
us['country'] = us['country'].str.replace('Michigan', 'MI')
us['country'] = us['country'].str.replace('Colorado', 'CO')
us['country'] = us['country'].str.replace('Oregon', 'OR')
us['country'] = us['country'].str.replace('Seattle', 'WA')
us['country'] = us['country'].str.replace('Boston', 'MA')
us['country'] = us['country'].str.replace('New Jersey', 'NJ')
us['country'] = us['country'].str.replace('Minnesota', 'MN')
us.loc[us['cityOrState'].str.contains('DC') | us['country'].str.contains('DC'), 'cityOrState'] = 'DC'
us.loc[us['cityOrState'].str.contains('DC') | us['country'].str.contains('DC'), 'country'] = 'USA'

In [18]:
condition = (us['country'] != 'USA') & (us['country'].str.len() != 2) & (us['country'].str.contains(r'[A-Z]{2}'))
us.loc[condition, ['country']] = us.loc[condition, 'country'].str.extract(r'([A-Z]{2})')
us['country'].value_counts(dropna=False)

USA                                     37819
CA                                       6451
FL                                       2563
IL                                       2365
TX                                       2161
MA                                       1674
WA                                       1584
PA                                       1379
MI                                       1081
OH                                       1067
CO                                       1050
NC                                       1017
GA                                       1009
OR                                        932
MN                                        865
NJ                                        823
VA                                        806
AZ                                        782
TN                                        718
MD                                        633
MO                                        564
NV                                

In [19]:
us[us['country'].isna()]

,cityOrState,country
2016,Chicago,NaN
2405,Boston,NaN
6688,CA,NaN
6879,PA,NaN
9042,CA,NaN
10003,Chicago,NaN
11903,Boston,NaN
18412,Seattle,NaN
19893,CA,NaN
19921,CA,NaN


In [20]:
us.loc[us['country'].isna(), 'country'] = us.loc[us['country'].isna(), 'cityOrState']

In [21]:
us['country'].value_counts(dropna=False)

USA                         37819
CA                           6528
FL                           2566
IL                           2366
TX                           2163
MA                           1674
WA                           1585
PA                           1380
MI                           1082
OH                           1067
CO                           1052
NC                           1017
GA                           1009
OR                            932
MN                            865
NJ                            826
VA                            806
AZ                            782
TN                            720
MD                            633
MO                            565
NV                            463
IN                            438
SC                            378
WI                            357
CT                            301
KY                            280
AL                            257
OK                            228
Virginia      

In [22]:
us.loc[us['country'].str.len() == 2, 'country'].value_counts(dropna=False)

CA    6528
FL    2566
IL    2366
TX    2163
MA    1674
WA    1585
PA    1380
MI    1082
OH    1067
CO    1052
NC    1017
GA    1009
OR     932
MN     865
NJ     826
VA     806
AZ     782
TN     720
MD     633
MO     565
NV     463
IN     438
SC     378
WI     357
CT     301
KY     280
AL     257
OK     228
NM     198
UT     193
      ... 
IA     178
NE     145
AR     134
NH     103
WV     103
HI     102
RI     101
ID     100
MS      92
ME      75
MT      59
AK      57
VT      55
DE      49
SD      41
ND      41
WY      36
Ca      32
Ga      24
Pa      15
Ma      11
Il       7
Wa       5
wa       2
pa       1
il       1
dc       1
Mn       1
✈️       1
ma       1
Name: country, Length: 61, dtype: int64

In [23]:
us.loc[us['country'].str.len() == 2, 'country'] = us.loc[us['country'].str.len() == 2, 'country'].str.upper()
us.loc[us['country'].str.len() == 2, 'country'].value_counts(dropna=False)

CA    6560
FL    2566
IL    2374
TX    2163
MA    1686
WA    1592
PA    1396
MI    1082
OH    1067
CO    1052
GA    1033
NC    1017
OR     932
MN     866
NJ     826
VA     806
AZ     782
TN     720
MD     633
MO     565
NV     463
IN     438
SC     378
WI     357
CT     301
KY     280
AL     257
OK     228
NM     198
UT     193
KS     183
IA     178
NE     145
AR     134
NH     103
WV     103
HI     102
RI     101
ID     100
MS      92
ME      75
MT      59
AK      57
VT      55
DE      49
SD      41
ND      41
WY      36
✈️       1
DC       1
Name: country, dtype: int64

In [24]:
us.loc[us['country'].str.len() == 2, 'cityOrState'] = us.loc[us['country'].str.len() == 2, 'country']
us.loc[us['country'].str.len() == 2, 'country'] = 'USA'
us['country'].value_counts(dropna=False)

USA                                     72286
Virginia                                  221
Arizona                                   218
North Carolina                            211
Georgia                                   189
Pennsylvania                              177
Indiana                                   169
Philadelphia                              165
Atlanta                                   153
Washington                                152
Wisconsin                                 143
Alabama                                   136
Tennessee                                 130
Missouri                                  124
Iowa                                      106
Louisiana                                  99
Oklahoma                                   96
Connecticut                                94
Kentucky                                   92
Utah                                       86
Kansas                                     84
Arkansas                          

In [25]:
us[us['country'] != 'USA']

,cityOrState,country
244,Wisconsin,Wisconsin
392,Tennessee,Tennessee
429,Alabama,Alabama
461,Pennsylvania,Pennsylvania
492,Indiana,Indiana
575,CA,International
599,Idaho,Idaho
623,Tulsa,Oklahoma
627,Shenandoah Valley,Virginia
673,Sheffield,Pennsylvania


In [26]:
us.loc[us['country'] != 'USA', 'cityOrState'] = us.loc[us['country'] != 'USA', 'country']

In [27]:
us['country'] = 'USA'
us['country'].value_counts(dropna=False)

USA    76079
Name: country, dtype: int64

## City/State

In [28]:
us['cityOrState'].value_counts(dropna=False)

CA                           11365
NaN                           9244
NY                            9187
DC                            3086
FL                            2569
IL                            2374
TX                            2167
MA                            1687
WA                            1595
PA                            1406
MI                            1083
OH                            1069
CO                            1053
GA                            1037
NC                            1025
Florida                        997
Texas                          994
OR                             932
MN                             869
NJ                             842
VA                             809
AZ                             782
TN                             723
Pennsylvania                   690
North Carolina                 690
MD                             634
Virginia                       596
MO                             566
Georgia             

In [29]:
us['cityOrState'] = us['cityOrState'].str.replace('Texas', 'TX')
us['cityOrState'] = us['cityOrState'].str.replace('Florida', 'FL')
us['cityOrState'] = us['cityOrState'].str.replace('North Carolina', 'NC')
us['cityOrState'] = us['cityOrState'].str.replace('Pennsylvania', 'PA')
us['cityOrState'] = us['cityOrState'].str.replace('Virginia', 'VA')
us['cityOrState'] = us['cityOrState'].str.replace('Georgia', 'GA')
us['cityOrState'] = us['cityOrState'].str.replace('Arizona', 'AZ')
us['cityOrState'] = us['cityOrState'].str.replace('New Jersey', 'NJ')
us['cityOrState'] = us['cityOrState'].str.replace('Michigan', 'MI')
us['cityOrState'] = us['cityOrState'].str.replace('Washington', 'WA')
us['cityOrState'] = us['cityOrState'].str.replace('Indiana', 'IN')
us['cityOrState'] = us['cityOrState'].str.replace('Ohio', 'OH')
us['cityOrState'] = us['cityOrState'].str.replace('Tennessee', 'TN')
us['cityOrState'] = us['cityOrState'].str.replace('Wisconsin', 'WI')
us['cityOrState'] = us['cityOrState'].str.replace('Colorado', 'CO')
us['cityOrState'] = us['cityOrState'].str.replace('Missouri', 'MO')
us['cityOrState'] = us['cityOrState'].str.replace('Alabama', 'AL')
us['cityOrState'] = us['cityOrState'].str.replace('Connecticut', 'CT')
us['cityOrState'].value_counts(dropna=False)

CA                           11365
NaN                           9244
NY                            9187
FL                            3566
TX                            3161
DC                            3086
IL                            2374
PA                            2096
WA                            2041
NC                            1715
MA                            1687
GA                            1557
MI                            1534
OH                            1444
VA                            1405
CO                            1393
NJ                            1335
AZ                            1298
TN                            1088
OR                             932
MO                             884
MN                             869
IN                             816
WI                             722
MD                             634
CT                             617
AL                             568
NV                             463
SC                  

In [30]:
condition = (us['cityOrState'].str.contains(r'[A-Z]{2}')) & (us['cityOrState'].str.len() != 2) & (us['cityOrState'].isna() == False)
us.loc[condition]

,cityOrState,country
689,Saxapahaw NC,USA
1451,"UK, and Belgium",USA
2003,NYC Metro Area,USA
3103,Bay Area CA,USA
4294,Sarasota FL,USA
4544,Panhandle of FL,USA
6328,CST,USA
6468,"Montana, IN",USA
6852,NOWHERE,USA
9673,Somewhere in CO,USA


In [31]:
us.loc[condition, ['cityOrState']] = us[condition]['cityOrState'].str.extract(r'([A-Z]{2})')
us['cityOrState'].value_counts(dropna=False)

CA                        11365
NaN                        9572
NY                         9187
FL                         3566
TX                         3161
DC                         3086
IL                         2374
PA                         2096
WA                         2041
NC                         1715
MA                         1687
GA                         1557
MI                         1534
OH                         1444
VA                         1405
CO                         1393
NJ                         1335
AZ                         1298
TN                         1088
OR                          932
MO                          884
MN                          869
IN                          816
WI                          722
MD                          634
CT                          617
AL                          568
NV                          463
SC                          380
Maryland                    283
                          ...  
Western 

In [32]:
us[us['cityOrState'].isin(states) == False]['cityOrState'].value_counts(dropna=False)

NaN                           9572
Maryland                       283
Illinois                       280
Oregon                         279
Kentucky                       256
Minnesota                      239
Iowa                           214
Oklahoma                       212
Louisiana                      196
South Carolina                 189
Philadelphia                   189
Kansas                         180
Utah                           177
Atlanta                        177
Arkansas                       173
New Mexico                     144
Nevada                         133
New Hampshire                  131
Maine                          118
Montana                        115
Idaho                          106
Vermont                        102
Nebraska                        94
Mississippi                     94
Hawaii                          88
Alaska                          79
Delaware                        77
Boston                          76
Rhode Island        

In [33]:
_ = us[us['cityOrState'].isin(states) == False]['cityOrState'].value_counts(dropna=False).index.tolist()[1:40]
_

['Maryland',
 'Illinois',
 'Oregon',
 'Kentucky',
 'Minnesota',
 'Iowa',
 'Oklahoma',
 'Louisiana',
 'South Carolina',
 'Philadelphia',
 'Kansas',
 'Utah',
 'Atlanta',
 'Arkansas',
 'New Mexico',
 'Nevada',
 'New Hampshire',
 'Maine',
 'Montana',
 'Idaho',
 'Vermont',
 'Nebraska',
 'Mississippi',
 'Hawaii',
 'Alaska',
 'Delaware',
 'Boston',
 'Rhode Island',
 'Chicago',
 'Wyoming',
 'Seattle',
 'District of Columbia',
 'South Dakota',
 'North Dakota',
 'Midwest',
 'Puerto Rico',
 'East Coast',
 'Everywhere',
 'New England']

In [34]:
repl = r"us['cityOrState'] = us['cityOrState'].str.replace("
repl

"us['cityOrState'] = us['cityOrState'].str.replace("

In [35]:
for i in sorted(_):
    print(f"{repl}'{i}', '')")

us['cityOrState'] = us['cityOrState'].str.replace('Alaska', '')
us['cityOrState'] = us['cityOrState'].str.replace('Arkansas', '')
us['cityOrState'] = us['cityOrState'].str.replace('Atlanta', '')
us['cityOrState'] = us['cityOrState'].str.replace('Boston', '')
us['cityOrState'] = us['cityOrState'].str.replace('Chicago', '')
us['cityOrState'] = us['cityOrState'].str.replace('Delaware', '')
us['cityOrState'] = us['cityOrState'].str.replace('District of Columbia', '')
us['cityOrState'] = us['cityOrState'].str.replace('East Coast', '')
us['cityOrState'] = us['cityOrState'].str.replace('Everywhere', '')
us['cityOrState'] = us['cityOrState'].str.replace('Hawaii', '')
us['cityOrState'] = us['cityOrState'].str.replace('Idaho', '')
us['cityOrState'] = us['cityOrState'].str.replace('Illinois', '')
us['cityOrState'] = us['cityOrState'].str.replace('Iowa', '')
us['cityOrState'] = us['cityOrState'].str.replace('Kansas', '')
us['cityOrState'] = us['cityOrState'].str.replace('Kentucky', '')
us['cityOrS

In [36]:
us['cityOrState'] = us['cityOrState'].str.replace('Alaska', 'AK')
us['cityOrState'] = us['cityOrState'].str.replace('Arkansas', 'AR')
us['cityOrState'] = us['cityOrState'].str.replace('Atlanta', 'GA')
us['cityOrState'] = us['cityOrState'].str.replace('Boston', 'MA')
us['cityOrState'] = us['cityOrState'].str.replace('Chicago', 'IL')
us['cityOrState'] = us['cityOrState'].str.replace('Delaware', 'DE')
us['cityOrState'] = us['cityOrState'].str.replace('District of Columbia', 'DC')
us['cityOrState'] = us['cityOrState'].str.replace('Hawaii', 'HI')
us['cityOrState'] = us['cityOrState'].str.replace('Idaho', 'ID')
us['cityOrState'] = us['cityOrState'].str.replace('Illinois', 'IL')
us['cityOrState'] = us['cityOrState'].str.replace('Iowa', 'IA')
us['cityOrState'] = us['cityOrState'].str.replace('Kansas', 'KS')
us['cityOrState'] = us['cityOrState'].str.replace('Kentucky', 'KY')
us['cityOrState'] = us['cityOrState'].str.replace('Louisiana', 'LA')
us['cityOrState'] = us['cityOrState'].str.replace('Maine', 'ME')
us['cityOrState'] = us['cityOrState'].str.replace('Maryland', 'MD')
us['cityOrState'] = us['cityOrState'].str.replace('Minnesota', 'MN')
us['cityOrState'] = us['cityOrState'].str.replace('Mississippi', 'MS')
us['cityOrState'] = us['cityOrState'].str.replace('Montana', 'MT')
us['cityOrState'] = us['cityOrState'].str.replace('Nebraska', 'NE')
us['cityOrState'] = us['cityOrState'].str.replace('Nevada', 'NV')
us['cityOrState'] = us['cityOrState'].str.replace('New England', 'NH')
us['cityOrState'] = us['cityOrState'].str.replace('New Hampshire', 'NH')
us['cityOrState'] = us['cityOrState'].str.replace('New Mexico', 'NM')
us['cityOrState'] = us['cityOrState'].str.replace('North Dakota', 'ND')
us['cityOrState'] = us['cityOrState'].str.replace('Oklahoma', 'OK')
us['cityOrState'] = us['cityOrState'].str.replace('Oregon', 'OR')
us['cityOrState'] = us['cityOrState'].str.replace('Philadelphia', 'PA')
us['cityOrState'] = us['cityOrState'].str.replace('Rhode Island', 'RI')
us['cityOrState'] = us['cityOrState'].str.replace('Seattle', 'WA')
us['cityOrState'] = us['cityOrState'].str.replace('South Carolina', 'SC')
us['cityOrState'] = us['cityOrState'].str.replace('South Dakota', 'SD')
us['cityOrState'] = us['cityOrState'].str.replace('Utah', 'UT')
us['cityOrState'] = us['cityOrState'].str.replace('Vermont', 'VT')
us['cityOrState'] = us['cityOrState'].str.replace('Wyoming', 'WY')

In [37]:
us.loc[us['cityOrState'] == 'Puerto Rico', 'country'] = 'Puerto Rico'
us[us['cityOrState'].isin(states) == False]['cityOrState'].value_counts(dropna=False)

NaN                           9572
Midwest                         26
Puerto Rico                     24
East Coast                      19
Everywhere                      18
Northern California             14
Mass                            12
Anytown                         12
                                10
SoCal                            9
Pacific Northwest                8
Cambridge                        8
Anywhere                         8
Virgin Islands                   7
Somewhere                        7
West Coast                       7
Earth                            6
Southeast                        6
Ill                              6
Harlem                           6
Upstate New York                 5
Nunya                            4
Left Coast                       4
Baltimore                        4
mostly                           4
Shithole                         4
America                          4
ILland                           4
Crazytown           

In [38]:
condition = (us['cityOrState'].isin(states) == False) & (us['cityOrState'] != 'Puerto Rico')
us.loc[condition, 'cityOrState'] = np.nan

In [39]:
us[us['cityOrState'].isin(states) == False]['cityOrState'].value_counts(dropna=False)

NaN            10512
Puerto Rico       24
Name: cityOrState, dtype: int64

In [40]:
us['cityOrState'].value_counts(dropna=False)

CA             11365
NaN            10512
NY              9187
FL              3566
TX              3161
DC              3126
IL              2720
PA              2285
WA              2084
MA              1763
GA              1734
NC              1715
MI              1534
OH              1444
VA              1405
CO              1393
NJ              1335
AZ              1298
OR              1211
MN              1108
TN              1088
MD               917
MO               884
IN               816
WI               722
CT               617
NV               596
SC               569
AL               568
KY               538
OK               440
IA               392
UT               371
KS               363
NM               342
AR               307
NH               251
NE               239
ID               206
LA               196
ME               193
HI               190
MS               186
RI               177
MT               174
VT               157
AK               136
DE           

## Non-US

In [41]:
notus

,cityOrState,country
0,Quelque part,None
1,London (E17) / Gateshead (NE9),None
2,add my snapchat - DirectPorn,None
3,NaN,NaN
4,NaN,NaN
5,Nationwide,None
6,NaN,NaN
8,NaN,NaN
12,West of the Rocky Mountains,None
16,Cardiff,UK


In [42]:
notus['cityOrState'].value_counts(dropna=False)

NaN                               55126
London                             2347
Toronto                            1521
Canada                              911
Earth                               520
Sydney                              499
Australia                           493
Melbourne                           434
Vancouver                           433
UK                                  423
England                             415
United Kingdom                      407
India                               382
Ottawa                              321
Stockholm                           295
Ontario                             288
Paris                               279
Worldwide                           258
Global                              255
Everywhere                          242
Planet Earth                        233
Berlin                              205
Mumbai                              203
New Delhi                           202
Dublin                              201


In [43]:
notus['country'].value_counts(dropna=False)

NaN                           111645
England                         2024
Ontario                         1149
Canada                           874
India                            723
UK                               564
Australia                        517
United Kingdom                   462
France                           346
Ireland                          300
South Africa                     299
British Columbia                 286
Germany                          270
Alberta                          256
Scotland                         251
Sweden                           220
Brasil                           215
New South Wales                  204
Victoria                         194
ON                               174
BC                               165
Ontario, Canada                  161
London                           153
Québec                           153
Nigeria                          138
Spain                            121
The Netherlands                  119
P

In [44]:
condition = (notus['cityOrState'].isna() == False) & (notus['country'].isna() == True)
notus[condition]

,cityOrState,country
0,Quelque part,None
1,London (E17) / Gateshead (NE9),None
2,add my snapchat - DirectPorn,None
5,Nationwide,None
12,West of the Rocky Mountains,None
18,Stuck in the Mud,None
20,au where seokjin has a movie coming out and ha...,None
21,@kerupukaleng,None
25,Austria,None
29,Tara Follows,None


In [45]:
notus.loc[condition, 'country'] = notus.loc[condition, 'cityOrState']

In [46]:
notus['country'].value_counts(dropna=False)[:45]

NaN                         55126
England                      2138
Canada                       1760
London                       1186
Ontario                      1176
India                        1100
Australia                    1000
UK                            972
United Kingdom                864
Toronto                       511
Earth                         479
France                        469
Ireland                       465
South Africa                  451
Germany                       441
Scotland                      363
Sweden                        363
British Columbia              304
Brasil                        295
Alberta                       271
Worldwide                     257
Global                        252
Everywhere                    240
Planet Earth                  224
Maryland                      210
New South Wales               205
The Netherlands               204
Victoria                      198
New Zealand                   192
Nigeria       

In [47]:
notus.isna().sum()

cityOrState    55126
country        55126
dtype: int64

In [48]:
_ = notus['country'].value_counts(dropna=False).index.tolist()[1:45]
_

['England',
 'Canada',
 'London',
 'Ontario',
 'India',
 'Australia',
 'UK',
 'United Kingdom',
 'Toronto',
 'Earth',
 'France',
 'Ireland',
 'South Africa',
 'Germany',
 'Scotland',
 'Sweden',
 'British Columbia',
 'Brasil',
 'Alberta',
 'Worldwide',
 'Global',
 'Everywhere',
 'Planet Earth',
 'Maryland',
 'New South Wales',
 'The Netherlands',
 'Victoria',
 'New Zealand',
 'Nigeria',
 'Brazil',
 'ON',
 'Pakistan',
 'Spain',
 'BC',
 'Sydney',
 'México',
 'Ontario, Canada',
 'Québec',
 'Japan',
 'Kenya',
 'Italy',
 'United States of America',
 'Argentina',
 'Sverige']

In [49]:
repl = r"notus['country'] = notus['country'].str.replace("
repl

"notus['country'] = notus['country'].str.replace("

In [50]:
for i in sorted(_):
    print(f"{repl}'{i}', '')")

notus['country'] = notus['country'].str.replace('Alberta', '')
notus['country'] = notus['country'].str.replace('Argentina', '')
notus['country'] = notus['country'].str.replace('Australia', '')
notus['country'] = notus['country'].str.replace('BC', '')
notus['country'] = notus['country'].str.replace('Brasil', '')
notus['country'] = notus['country'].str.replace('Brazil', '')
notus['country'] = notus['country'].str.replace('British Columbia', '')
notus['country'] = notus['country'].str.replace('Canada', '')
notus['country'] = notus['country'].str.replace('Earth', '')
notus['country'] = notus['country'].str.replace('England', '')
notus['country'] = notus['country'].str.replace('Everywhere', '')
notus['country'] = notus['country'].str.replace('France', '')
notus['country'] = notus['country'].str.replace('Germany', '')
notus['country'] = notus['country'].str.replace('Global', '')
notus['country'] = notus['country'].str.replace('India', '')
notus['country'] = notus['country'].str.replace('Irel

In [51]:
canada = ['Alberta', 'BC', 'British Columbia', 'ON', 'Ontario', 'Ontario, Canada', 'Toronto', 'Québec']
notus.loc[notus['country'].isin(canada)]

,cityOrState,country
17,Fort McMurray,Alberta
152,Vancouver,British Columbia
219,Kitchener,Ontario
225,Kingston,Ontario
385,Ottawa,"Ontario, Canada"
405,Toronto,Ontario
630,Kincardine,Ontario
665,Waterloo,Ontario
691,Langley,British Columbia
758,Thunder Bay,Ontario


In [52]:
notus.loc[notus['country'].isin(canada), 'country'].value_counts(dropna=False)

Ontario             1176
Toronto              511
British Columbia     304
Alberta              271
ON                   175
BC                   170
Ontario, Canada      161
Québec               160
Name: country, dtype: int64

In [53]:
notus.loc[notus['country'].isin(canada), 'country'] = 'Canada'
notus.loc[notus['country'].str.contains('Canada', case=False) == True, 'country'] = 'Canada'
notus.loc[(notus['cityOrState'] == 'Canada') & (notus['country'] == 'Canada'), 'cityOrState'] = np.nan

In [54]:
condition = (notus['cityOrState'].str.contains('Canada', case=False)) & (notus['country'] == 'Canada')
notus.loc[condition]

,cityOrState,country
605,Canada eh,Canada
799,Ontario Canada,Canada
912,Ontario Canada,Canada
1803,CANADA,Canada
3200,Italy / UK / Canada,Canada
3920,Western Canada Zone,Canada
4082,canada,Canada
4254,Toronto - Canada,Canada
4328,Canada & Japan,Canada
4856,Regina Saskatchewan CANADA,Canada


In [55]:
notus.loc[notus['country'] == 'Canada', 'cityOrState'].value_counts()

Toronto                           1477
Vancouver                          303
Ontario                            285
Ottawa                             259
Montréal                           145
Calgary                            139
Edmonton                           135
British Columbia                    90
Alberta                             78
London                              72
Hamilton                            60
Victoria                            56
Montreal                            40
Nova Scotia                         35
Québec                              31
canada                              30
Mississauga                         27
Kingston                            27
Waterloo                            22
Guelph                              21
Kitchener                           19
Halifax                             19
Winnipeg                            18
Windsor                             17
Surrey                              17
Ontario Canada           

In [56]:
uk = ['London', 'United Kingdom', 'England']
notus.loc[notus['country'].isin(uk), 'country'] = 'UK'
notus.loc[notus['cityOrState'].isin(uk), 'country'] = 'UK'
notus.loc[notus['country'] == 'UK', 'cityOrState'].value_counts(dropna=False)

London                   2347
England                   415
UK                        409
United Kingdom            407
Manchester                107
Scotland                   76
Bristol                    57
Birmingham                 47
Sheffield                  45
South East                 43
Leeds                      42
Liverpool                  41
West Midlands              34
North West                 34
Wales                      34
Cambridge                  34
Oxford                     32
Nottingham                 30
Brighton                   30
Newcastle Upon Tyne        26
South West                 23
City of London             21
North East                 21
Leicester                  20
Hackney                    17
Southampton                15
Bath                       15
East                       14
Reading                    14
Norwich                    13
                         ... 
Ilford                      1
Knightsbridge               1
Docklands 

In [57]:
aus = ['Sydney', 'New South Wales', 'Australia', 'Melbourne', 'Brisbane', 'Queensland', 'Canberra', 'Perth']
notus.loc[notus['country'].isin(aus), 'country'] = 'Australia'
notus.loc[notus['cityOrState'].isin(aus), 'country'] = 'Australia'
notus.loc[notus['country'] == 'Australia', 'cityOrState'].value_counts(dropna=False)

Sydney                          499
Australia                       493
Melbourne                       434
Brisbane                        119
Perth                            78
Canberra                         64
Queensland                       36
Victoria                         29
New South Wales                  21
Gold Coast                       18
Newcastle                        13
Wollongong                       10
Sunshine Coast                    9
Adelaide                          8
Tasmania                          7
Byron Bay                         7
Central Coast                     7
NSW                               6
Townsville                        4
Hobart                            3
Western Australia                 3
Wagga Wagga                       3
Cairns                            3
Toowoomba                         2
Orange                            2
Northern Territory                2
Port Melbourne                    2
42 Wallaby Way              

In [58]:
brazil = ['Brasil', 'Brazil', 'Brasília', 'Brasilia', 'São Paulo', 'Sao Paulo']
notus.loc[notus['country'].isin(brazil), 'country'] = 'Brazil'
notus.loc[notus['cityOrState'].isin(brazil), 'country'] = 'Brazil'
notus.loc[notus['cityOrState'].isin(['Brasil', 'Brazil']), 'cityOrState'] = 'Brazil'
notus.loc[notus['cityOrState'].isin(['São Paulo', 'Sao Paulo']), 'cityOrState'] = 'Sao Paulo'
notus.loc[notus['country'] == 'Brazil', 'cityOrState'].value_counts(dropna=False)

Brazil                   150
Sao Paulo                105
Rio de Janeiro            51
Curitiba                   9
Fortaleza                  8
Brasília                   8
Belo Horizonte             8
Goiânia                    7
Minas Gerais               7
Porto Alegre               7
Campinas                   6
Recife                     6
Manaus                     5
Rio Grande do Sul          5
Bahia                      4
Pernambuco                 4
Santa Catarina             4
Ceará                      3
Teresina                   3
Salvador                   3
Florianópolis              3
Federal District           2
Paraná                     2
Sorocaba                   2
Belém                      2
Cuiabá                     2
Santos                     2
Duque de Caxias            2
Natal                      2
Campos dos Goytacazes      2
                        ... 
Mossoró                    1
Aracati                    1
Uruguaiana                 1
Franca        

In [59]:
sweden = ['Sverige', 'Sweden', 'Stockholm', 'Gothenburg', 'Schweden', 'Sweden, Earth', 'Sweden, Universe']
notus.loc[notus['country'].isin(sweden), 'country'] = 'Sweden'
notus.loc[notus['cityOrState'].isin(sweden), 'country'] = 'Sweden'
notus.loc[notus['country'] == 'Sweden', 'cityOrState'].value_counts(dropna=False)

Stockholm          295
Sweden             146
Sverige             31
Gothenburg          17
Malmo                9
Uppsala              6
Göteborg             6
Helsingborg          6
Malmö                5
Lund                 5
Västerås             5
Umeå                 4
Linköping            4
Solna                3
Karlstad             3
Örebro               3
Sundbyberg           3
Gävle                3
Piteå                2
Nacka                2
Borås                2
Växjö                2
Norrköping           2
Jönköping            2
Sundsvall            2
Tyresö               2
Haninge              2
Enköping             2
Järfälla             2
lesbian islands      1
                  ... 
Anundgård            1
Askersund            1
Trollhättan          1
Täby                 1
Hedemora             1
Alingsås             1
Scania-3             1
Simrishamn           1
Luleå                1
Sollentuna           1
Värmdö               1
Bollnäs              1
Örebro Cast

In [60]:
notus['country'] = notus['country'].str.replace('United States of America', 'USA')
notus['country'] = notus['country'].str.replace('Maryland', 'USA')

In [61]:
mexico = ['Mexico', 'México', 'Distrito Federal', 'Puebla', 'Aguascalientes', 'San Luis Potosí']
notus.loc[notus['country'].isin(mexico), 'country'] = 'Mexico'
notus.loc[notus['cityOrState'].isin(mexico), 'country'] = 'Mexico'
notus.loc[notus['country'] == 'Mexico', 'cityOrState'].value_counts(dropna=False)

México                    95
Mexico                    92
Distrito Federal          11
Aguascalientes             7
Puebla                     6
San Luis Potosí            5
Baja California Sur        3
Oaxaca                     3
Jalisco                    3
Guadalajara                3
Tabasco                    2
Monterrey                  2
Querétaro                  2
Coyoacán                   2
Sinaloa                    2
Guanajuato                 2
Hidalgo                    2
Baja California            2
Pachuca                    2
Tamaulipas                 2
Acapulco                   2
Mexico City                2
Nuevo León                 2
Mérida                     1
Cancún                     1
Urbe de Humo               1
Cancun                     1
California & SMA           1
La Paz                     1
Querétaro Arteaga          1
Colima                     1
Ecatepec de Morelos        1
Tijuana                    1
Quintana Roo               1
Chihuahua     

In [62]:
jpn = ['Japan', 'Tokyo', 'JAPAN', 'Tokyo-to', 'Osaka', 'Okinawa', 'Chiba', 'Okinawa-ken', 'Kyoto', 'Miyazaki']
notus.loc[notus['country'].isin(jpn), 'country'] = 'Japan'
notus.loc[notus['cityOrState'].isin(jpn), 'country'] = 'Japan'
notus.loc[notus['country'] == 'Japan', 'cityOrState'].value_counts(dropna=False)

Japan                      68
Tokyo                      68
Tokyo-to                   18
Osaka                       9
JAPAN                       7
Kyoto                       6
Okinawa                     5
Chiba                       4
Okinawa-ken                 3
東京都 / Tokyo                 2
Nagoya                      2
Miyazaki                    2
Hokkaido                    2
Tsukuba                     1
Saga & Fukuoka              1
Danktown                    1
Adachi-ku                   1
Funabashi-shi               1
Nakano - Aoyama             1
Mie-ken                     1
Kyoto City Fushimi Ward     1
Shibuya-ku                  1
Kyoto City                  1
Aichi-ken                   1
Gunma-ken                   1
Kumamoto                    1
Yamanashi                   1
Hiroshima                   1
Hawaii / Kamakura           1
Saitama                     1
A city saved by magic       1
Chiba-shi Midori            1
Ibaraki-shi                 1
Shinjuku  

In [63]:
india = ['India', 'india', 'INDIA', 'India.', 'New Delhi', 'Mumbai', 
         'Bengaluru', 'Pune', 'Hyderabad', 'Kolkata', 'Chennai', 'Bangalore', 'Delhi']

notus.loc[notus['country'].isin(india), 'country'] = 'India'
notus.loc[notus['cityOrState'].isin(india), 'country'] = 'India'
notus.loc[notus['country'] == 'India', 'cityOrState'].value_counts(dropna=False)

India                                 382
Mumbai                                203
New Delhi                             202
Bengaluru                              67
Pune                                   50
Bangalore                              50
Hyderabad                              49
Delhi                                  42
Kolkata                                29
Chennai                                28
india                                  13
INDIA                                  12
Noida                                  10
Bengaluru South                        10
Gurgaon                                 9
Chandigarh                              9
Jaipur                                  8
Maharashtra                             7
Kerala                                  6
Nagpur                                  6
Ahmadabad City                          6
Uttar Pradesh                           6
Bhubaneshwar                            6
Thane                             

In [64]:
country = ['Argentina', 'Kenya', 'Nigeria', 'South Africa', 'France', 'Germany',
           'Scotland', 'Spain', 'Ireland', 'Italy', 'New Zealand', 'Pakistan']

for i in country:
    notus.loc[notus['country'] == i, 'country'] = i
    notus.loc[notus['cityOrState'] == i, 'country'] = i

In [65]:
notus.loc[notus['cityOrState'] == 'Londo', 'cityOrState'] = 'London'
notus['cityOrState'] = notus['cityOrState'].str.strip('\n')
notus['cityOrState'] = notus['cityOrState'].str.strip()

In [66]:
len(us) + len(notus) == len(geo)

True

In [67]:
newgeo = pd.concat([us, notus])
len(geo) == len(newgeo)

True

In [68]:
newgeo.sort_index(inplace=True)

In [69]:
users = users.join(newgeo)

In [70]:
users.columns.tolist()

['screenName',
 'id_str',
 'name',
 'screen_name',
 'location',
 'description',
 'url',
 'followers_count',
 'friends_count',
 'created_at',
 'favourites_count',
 'verified',
 'statuses_count',
 'lang',
 'cityOrState',
 'country']

In [71]:
users.head()

,screenName,id_str,name,screen_name,location,description,url,followers_count,friends_count,created_at,favourites_count,verified,statuses_count,lang,cityOrState,country
0,rougbaisers,190672282,j’aime,rougbaisers,Quelque part,"Je suis ici avec vous, pour l'instant",NaN,1723,2417,Tue Sep 14 15:10:30 +0000 2010,1747,False,204451,en,Quelque part,Quelque part
1,CalumSPlath,61073310,Calum Sherwood,CalumSPlath,London (E17) / Gateshead (NE9),Sylvia Plath enthusiast. Originally from the N...,https://t.co/yELmZKm2Y0,5708,2176,Wed Jul 29 01:10:37 +0000 2009,71775,False,99881,en,London (E17) / Gateshead (NE9),London (E17) / Gateshead (NE9)
2,scfuckers,880227723494477829,Scfuckers.com,scfuckers,add my snapchat - DirectPorn,visit https://t.co/EYipnQWzZl if you are serio...,https://t.co/5qswIAS5fJ,13462,0,Thu Jun 29 00:53:37 +0000 2017,0,False,43457,en,add my snapchat - DirectPorn,add my snapchat - DirectPorn
3,shoegal27,20514782,Karina Thomas,shoegal27,NaN,B2B marketer and part time #DDMIX instructor w...,https://t.co/ladqhZ5Dyw,174,392,Tue Feb 10 14:04:49 +0000 2009,1773,False,688,en,NaN,NaN
4,BasedHowardBeal,888919789065973761,Jeremy Maddux,BasedHowardBeal,NaN,"Host of The Quiet Place\n\nChaotic Neutral, De...",NaN,138,736,Sun Jul 23 00:32:47 +0000 2017,4544,False,4202,en,NaN,NaN


In [72]:
cols = ['screenName', 'id_str', 'name', 'description', 'url',
        'location', 'cityOrState', 'country', 'lang', 'created_at', 'verified',
        'followers_count', 'friends_count', 'favourites_count', 'statuses_count']

users = users[cols]

In [73]:
condition = (users['cityOrState'].isna() == False) & (users['country'].isna() == False)
users[condition].head()

,screenName,id_str,name,description,url,location,cityOrState,country,lang,created_at,verified,followers_count,friends_count,favourites_count,statuses_count
0,rougbaisers,190672282,j’aime,"Je suis ici avec vous, pour l'instant",NaN,Quelque part,Quelque part,Quelque part,en,Tue Sep 14 15:10:30 +0000 2010,False,1723,2417,1747,204451
1,CalumSPlath,61073310,Calum Sherwood,Sylvia Plath enthusiast. Originally from the N...,https://t.co/yELmZKm2Y0,London (E17) / Gateshead (NE9),London (E17) / Gateshead (NE9),London (E17) / Gateshead (NE9),en,Wed Jul 29 01:10:37 +0000 2009,False,5708,2176,71775,99881
2,scfuckers,880227723494477829,Scfuckers.com,visit https://t.co/EYipnQWzZl if you are serio...,https://t.co/5qswIAS5fJ,add my snapchat - DirectPorn,add my snapchat - DirectPorn,add my snapchat - DirectPorn,en,Thu Jun 29 00:53:37 +0000 2017,False,13462,0,0,43457
5,SANEvents2017,368962215,Book Sophia Nelson,2018 is here! It's time to Go from your Now to...,https://t.co/qUaJMvwrFX,Nationwide,Nationwide,Nationwide,en,Tue Sep 06 14:41:00 +0000 2011,False,1041,117,1947,12122
7,lucycarin,18765167,lucycarin,It isn't braggin if you can do it. I’m a new O...,NaN,"Ponca City, OK",OK,USA,en,Thu Jan 08 14:38:31 +0000 2009,False,1248,380,170422,112047


In [74]:
users.dtypes

screenName          object
id_str              object
name                object
description         object
url                 object
location            object
cityOrState         object
country             object
lang                object
created_at          object
verified            object
followers_count     object
friends_count       object
favourites_count    object
statuses_count      object
dtype: object

In [75]:
len(users)
users.drop(users[users['followers_count'] == 'followers_count'].index, inplace=True)
len(users)

211839

211838

In [76]:
users[users['friends_count'].str.contains(r'[A-Za-z]') == True]

,screenName,id_str,name,description,url,location,cityOrState,country,lang,created_at,verified,followers_count,friends_count,favourites_count,statuses_count
23047,I FEEL everything - will cry constantly 💖 She/Her,NaN,635,23218,False,Sun Mar 24 03:15:04 +0000 2013,Sun Mar 24 03:15:04 +0000 2013,Sun Mar 24 03:15:04 +0000 2013,NaN,NaN,NaN,7591.0,en,NaN,NaN
36251,Nr.1 Enemy of Russophobes. Pronouns: His Maje...,https://t.co/iSiimAwzUq,242,14364,False,Mon May 25 20:38:49 +0000 2015,Mon May 25 20:38:49 +0000 2015,Mon May 25 20:38:49 +0000 2015,NaN,NaN,NaN,7163.0,sv,NaN,NaN
38920,Blocked by the KimBriggsCampaign,NaN,345,10837,False,Sat Jul 09 11:16:50 +0000 2011,Sat Jul 09 11:16:50 +0000 2011,Sat Jul 09 11:16:50 +0000 2011,NaN,NaN,NaN,15521.0,en,NaN,NaN
52544,Contact us: clpsec@chichester-labour.org.uk,http://t.co/Vw1L5tquRw,638,4,False,Thu Jan 08 09:32:21 +0000 2015,Thu Jan 08 09:32:21 +0000 2015,Thu Jan 08 09:32:21 +0000 2015,NaN,NaN,NaN,7256.0,en,NaN,NaN
52749,#TheResistance #KnowledgeIsPower #TruthSeeker,NaN,2358,2862,False,Fri Apr 09 04:03:18 +0000 2010,Fri Apr 09 04:03:18 +0000 2010,Fri Apr 09 04:03:18 +0000 2010,NaN,NaN,NaN,125648.0,en,NaN,NaN
74663,Follow and Tag #DDCmusic for favs and retweets!,https://t.co/KQW77QeSKz,12923,19831,False,Sat Jun 28 20:18:30 +0000 2014,Sat Jun 28 20:18:30 +0000 2014,Sat Jun 28 20:18:30 +0000 2014,NaN,NaN,NaN,206755,en,NaN,NaN
79214,+1-360-441-2695,http://t.co/NeYiw7SATh,4408,22213,False,Tue Nov 21 02:35:17 +0000 2006,Tue Nov 21 02:35:17 +0000 2006,Tue Nov 21 02:35:17 +0000 2006,NaN,NaN,NaN,20337,en,NaN,NaN
90921,*Support Galway 2020*,NaN,3684,14482,False,Sun Jul 27 06:24:42 +0000 2014,Sun Jul 27 06:24:42 +0000 2014,Sun Jul 27 06:24:42 +0000 2014,NaN,NaN,NaN,314098,en,NaN,NaN
96094,#jazz #surf #Resister #bluewave2018,NaN,507,7133,False,Sun Apr 09 00:26:28 +0000 2017,Sun Apr 09 00:26:28 +0000 2017,Sun Apr 09 00:26:28 +0000 2017,NaN,NaN,NaN,3261,en,NaN,NaN
109523,#history @thinkUHI. All views are my own.,https://t.co/hZQbiHrdXq,901,28944,False,Tue May 10 14:58:00 +0000 2016,Tue May 10 14:58:00 +0000 2016,Tue May 10 14:58:00 +0000 2016,NaN,NaN,NaN,13028,en-gb,NaN,NaN


In [77]:
len(users)
users.drop(users[users['friends_count'].str.contains(r'[A-Za-z]') == True].index, inplace=True)
len(users)

211838

211815

In [78]:
users[users['friends_count'].isna()]

,screenName,id_str,name,description,url,location,cityOrState,country,lang,created_at,verified,followers_count,friends_count,favourites_count,statuses_count
551,Russian Nationalist.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23045,AbbiCaDaby,1293385314.0,Abbie 🌈,Just trying to spread the sunshine 🌞 Gay af 🌈 ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23046,Nature gal 🌼Animals are my friends 🐮🐷🐱,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36250,AntonAyx,3298412639.0,Anton,Russian Nationalist.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38918,TopHat_247,332192833,Toppy,Fat lad on a bike... Park Run enthusiast.,NaN,"London, England",London,UK,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38919,Suspects London cabbies may genuinely be the t...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52543,ChiLabour_15,2967541768,Chichester Labour,🌹More than 1 in 5 voters voted Labour in our c...,NaN,"Theatre House, Theatre Lane, Chichester PO19 1SS",Theatre House,"Theatre Lane, Chichester PO19 1SS",NaN,NaN,NaN,NaN,NaN,NaN,NaN
52748,chickilopiccolo,131056491,Lisa LoPiccolo 🗽🇺🇸🌎✌,Artist and Chef,NaN,Earth,Earth,Earth,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74661,DistrictDaCapo,2593600476,DistrictDaCapo.com,Everything #Indie #music,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74662,#FREE #promotion for #independent #musicians.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [79]:
len(users)
users.drop(users[users['friends_count'].isna()].index, inplace=True)
len(users)

211815

211782

In [80]:
users['verified'].value_counts(dropna=False)

False    137242
False     63445
True       9018
True       2077
Name: verified, dtype: int64

In [81]:
users.loc[users['verified'] == True, 'verified'] = 1
users.loc[users['verified'] == False, 'verified'] = 0
users.loc[users['verified'] == 'True', 'verified'] = 1
users.loc[users['verified'] == 'False', 'verified'] = 0

In [84]:
users['verified'].value_counts(dropna=False)

0    200687
1     11095
Name: verified, dtype: int64

In [87]:
users.head()

,screenName,id_str,name,description,url,location,cityOrState,country,lang,created_at,verified,followers_count,friends_count,favourites_count,statuses_count
0,rougbaisers,190672282,j’aime,"Je suis ici avec vous, pour l'instant",NaN,Quelque part,Quelque part,Quelque part,en,2010-09-14 15:10:30,0,1723,2417,1747,204451
1,CalumSPlath,61073310,Calum Sherwood,Sylvia Plath enthusiast. Originally from the N...,https://t.co/yELmZKm2Y0,London (E17) / Gateshead (NE9),London (E17) / Gateshead (NE9),London (E17) / Gateshead (NE9),en,2009-07-29 01:10:37,0,5708,2176,71775,99881
2,scfuckers,880227723494477829,Scfuckers.com,visit https://t.co/EYipnQWzZl if you are serio...,https://t.co/5qswIAS5fJ,add my snapchat - DirectPorn,add my snapchat - DirectPorn,add my snapchat - DirectPorn,en,2017-06-29 00:53:37,0,13462,0,0,43457
3,shoegal27,20514782,Karina Thomas,B2B marketer and part time #DDMIX instructor w...,https://t.co/ladqhZ5Dyw,NaN,NaN,NaN,en,2009-02-10 14:04:49,0,174,392,1773,688
4,BasedHowardBeal,888919789065973761,Jeremy Maddux,"Host of The Quiet Place\n\nChaotic Neutral, De...",NaN,NaN,NaN,NaN,en,2017-07-23 00:32:47,0,138,736,4544,4202


In [88]:
users.dtypes

screenName                  object
id_str                      object
name                        object
description                 object
url                         object
location                    object
cityOrState                 object
country                     object
lang                        object
created_at          datetime64[ns]
verified                     int64
followers_count             object
friends_count               object
favourites_count            object
statuses_count              object
dtype: object

In [95]:
users['created_at'] = pd.to_datetime(users['created_at'])
users['followers_count'] = pd.to_numeric(users['followers_count'], downcast='integer')
users['friends_count'] = pd.to_numeric(users['friends_count'], downcast='integer')
users['favourites_count'] = pd.to_numeric(users['favourites_count'], downcast='integer')
users['statuses_count'] = pd.to_numeric(users['statuses_count'], downcast='integer')

In [97]:
len(users)
users.drop_duplicates(subset='screenName', keep='last', inplace=True)
users.reset_index(inplace=True, drop=True)
len(users)

211782

211782

In [98]:
users.head()

,screenName,id_str,name,description,url,location,cityOrState,country,lang,created_at,verified,followers_count,friends_count,favourites_count,statuses_count
0,rougbaisers,190672282,j’aime,"Je suis ici avec vous, pour l'instant",NaN,Quelque part,Quelque part,Quelque part,en,2010-09-14 15:10:30,0,1723,2417,1747,204451
1,CalumSPlath,61073310,Calum Sherwood,Sylvia Plath enthusiast. Originally from the N...,https://t.co/yELmZKm2Y0,London (E17) / Gateshead (NE9),London (E17) / Gateshead (NE9),London (E17) / Gateshead (NE9),en,2009-07-29 01:10:37,0,5708,2176,71775,99881
2,scfuckers,880227723494477829,Scfuckers.com,visit https://t.co/EYipnQWzZl if you are serio...,https://t.co/5qswIAS5fJ,add my snapchat - DirectPorn,add my snapchat - DirectPorn,add my snapchat - DirectPorn,en,2017-06-29 00:53:37,0,13462,0,0,43457
3,shoegal27,20514782,Karina Thomas,B2B marketer and part time #DDMIX instructor w...,https://t.co/ladqhZ5Dyw,NaN,NaN,NaN,en,2009-02-10 14:04:49,0,174,392,1773,688
4,BasedHowardBeal,888919789065973761,Jeremy Maddux,"Host of The Quiet Place\n\nChaotic Neutral, De...",NaN,NaN,NaN,NaN,en,2017-07-23 00:32:47,0,138,736,4544,4202


In [ ]:
users.drop(users[users['screenName'] == 'scfuckers'].index, 0, inplace=True)

In [99]:
users.to_csv('Data/users3.csv', index=False, encoding='utf-8')

# Tweets

In [108]:
tweets.head()

,tweetId,screenName,text,replyToSN,isRetweet,retweeted,retweetCount,favorited,favoriteCount,truncated,created,statusSource,longitude,latitude
0,1,ahmediaTV,American Harem.. #MeToo https://t.co/HjExLJdGuF,NaN,False,False,0,False,0,False,2017-11-29 23:59:00,"<a href=""http://instagram.com"" rel=""nofollow"">...",NaN,NaN
1,2,JesusPrepper74,@johnconyersjr @alfranken why have you guys ...,johnconyersjr,False,False,0,False,0,False,2017-11-29 23:59:00,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",NaN,NaN
2,3,DemerisePotvin,Watched Megan Kelly ask Joe Keery this A.M. if...,NaN,False,False,0,False,0,True,2017-11-29 23:59:00,"<a href=""http://twitter.com/download/android"" ...",NaN,NaN
3,4,TheDawnStott,Women have been talking about this crap the en...,NaN,False,False,0,False,0,False,2017-11-29 23:59:00,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",NaN,NaN
4,5,scottygirl2014,.@BetteMidler please speak to this sexual assa...,NaN,False,False,11,False,15,False,2017-11-29 23:59:00,"<a href=""http://twitter.com/#!/download/ipad"" ...",NaN,NaN


In [109]:
tweets.dtypes

tweetId            int64
screenName        object
text              object
replyToSN         object
isRetweet           bool
retweeted           bool
retweetCount       int64
favorited           bool
favoriteCount      int64
truncated           bool
created           object
statusSource      object
longitude        float64
latitude         float64
dtype: object

In [110]:
tweets['created'] = pd.to_datetime(tweets['created'])

In [111]:
tweets['retweeted'].value_counts(dropna=False)
tweets['favorited'].value_counts(dropna=False)

False    390288
Name: retweeted, dtype: int64

False    390288
Name: favorited, dtype: int64

In [112]:
tweets.drop(['retweeted','favorited'],1,inplace=True)

In [113]:
tweets['isRetweet'].value_counts(dropna=False)
tweets['truncated'].value_counts(dropna=False)

True     224702
False    165586
Name: isRetweet, dtype: int64

False    304715
True      85573
Name: truncated, dtype: int64

In [116]:
tweets.loc[tweets['isRetweet'] == True, 'isRetweet'] = 1
tweets.loc[tweets['isRetweet'] == False, 'isRetweet'] = 0
tweets.loc[tweets['truncated'] == True, 'truncated'] = 1
tweets.loc[tweets['truncated'] == False, 'truncated'] = 0

In [117]:
tweets['isRetweet'].value_counts(dropna=False)
tweets['truncated'].value_counts(dropna=False)

1    224702
0    165586
Name: isRetweet, dtype: int64

0    304715
1     85573
Name: truncated, dtype: int64

In [118]:
tweets.head()
tweets.dtypes

,tweetId,screenName,text,replyToSN,isRetweet,retweetCount,favoriteCount,truncated,created,statusSource,longitude,latitude
0,1,ahmediaTV,American Harem.. #MeToo https://t.co/HjExLJdGuF,NaN,0,0,0,0,2017-11-29 23:59:00,"<a href=""http://instagram.com"" rel=""nofollow"">...",NaN,NaN
1,2,JesusPrepper74,@johnconyersjr @alfranken why have you guys ...,johnconyersjr,0,0,0,0,2017-11-29 23:59:00,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",NaN,NaN
2,3,DemerisePotvin,Watched Megan Kelly ask Joe Keery this A.M. if...,NaN,0,0,0,1,2017-11-29 23:59:00,"<a href=""http://twitter.com/download/android"" ...",NaN,NaN
3,4,TheDawnStott,Women have been talking about this crap the en...,NaN,0,0,0,0,2017-11-29 23:59:00,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",NaN,NaN
4,5,scottygirl2014,.@BetteMidler please speak to this sexual assa...,NaN,0,11,15,0,2017-11-29 23:59:00,"<a href=""http://twitter.com/#!/download/ipad"" ...",NaN,NaN


tweetId                   int64
screenName               object
text                     object
replyToSN                object
isRetweet                 int64
retweetCount              int64
favoriteCount             int64
truncated                 int64
created          datetime64[ns]
statusSource             object
longitude               float64
latitude                float64
dtype: object

In [121]:
cols = ['tweetId', 'screenName', 'text', 'replyToSN', 'isRetweet', 'truncated', 'retweetCount', 'favoriteCount', 
        'created', 'longitude', 'latitude']
tweets = tweets[cols]

In [122]:
tweets.dtypes

tweetId                   int64
screenName               object
text                     object
replyToSN                object
isRetweet                 int64
truncated                 int64
retweetCount              int64
favoriteCount             int64
created          datetime64[ns]
longitude               float64
latitude                float64
dtype: object

In [123]:
tweets.to_csv('Data/tweets2.csv', index=False, encoding='utf-8')

# Hashtags

In [124]:
hashtags['hashtag'] = hashtags['hashtag'].str.replace(r'\\n',' ')
hashtags['hashtag'] = hashtags['hashtag'].str.replace('trump','Trump')
hashtags['hashtag'] = hashtags['hashtag'].str.replace('TRUMP','Trump')
hashtags['hashtag'] = hashtags['hashtag'].str.replace('sexualharassment','SexualHarassment')
hashtags['hashtag'] = hashtags['hashtag'].str.replace(r'M[Ee][Tt]oo','MeToo')
hashtags['hashtag'] = hashtags['hashtag'].str.replace('METOO','MeToo')
hashtags['hashtag'] = hashtags['hashtag'].str.replace('metoo','MeToo')

In [125]:
hashtags.to_csv('Data/hashtags.csv', index=False)